## Import Libraries

In [1]:
import pandas as pd
import lonely_boy2 as lb2
from sklearn import preprocessing
import numpy as np
from tqdm import tqdm_notebook,tqdm

## Initialize Relation Attributes

In [2]:
usr_ratings_cols = 'user id | item id | rating | timestamp'.split(' | ')
movie_info_cols = '''movie id | movie title | release date | video release date | IMDb URL | unknown | Action | Adventure | Animation | Children's | Comedy | Crime | Documentary | Drama | Fantasy | Film-Noir | Horror | Musical | Mystery | Romance | Sci-Fi | Thriller | War | Western'''.split(' | ')
usr_info_cols = 'user id | age | gender | occupation | zip code'.split(' | ')

## Initialize Relation URL's

In [3]:
usr_ratings_url = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.data'
movie_info_url  = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.item'
usr_info_url    = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.user'
genre_info_url  = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.genre'
ocp_info_url    = 'http://files.grouplens.org/datasets/movielens/ml-100k/u.occupation'

## Read CSV's

In [4]:
ratings = pd.read_csv(usr_ratings_url, delimiter='\t', header=None, names=usr_ratings_cols, encoding='latin-1')

movies = pd.read_csv(movie_info_url, delimiter='|', header=None, names=movie_info_cols, encoding='latin-1')

users = pd.read_csv(usr_info_url, delimiter='|', header=None, names=usr_info_cols, encoding='latin-1')

genres = pd.read_csv(genre_info_url, delimiter='|', header=None, names=['genre_id', 'genre_code'], encoding='latin-1')
genres = genres[['genre_id']]

ocps = pd.read_csv(ocp_info_url, delimiter='|', header=None, names=['ocp_id'], encoding='latin-1')
ocps = ocps[['ocp_id']]

In [5]:
users.head()

,user id,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [51]:
ratings_full

,user id,rating,movie id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,3,242,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,63,3,242,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,226,5,242,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,154,3,242,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,306,5,242,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,296,4,242,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6,34,5,242,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
7,271,4,242,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
8,201,4,242,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,209,4,242,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
ratings_full = ratings.merge(movies, left_on='item id', right_on='movie id')
ratings_full = ratings_full.drop(['item id','timestamp','movie title','release date','video release date','IMDb URL'],axis = 1)
ratings_full.head()


for index , value in tqdm_notebook(ratings_full.iterrows()):
    if value.rating < 3 : ratings_full.drop(index, inplace=True)

A Jupyter Widget

In [20]:
ratings_full3 = ratings_full.drop(['rating','movie id'],axis =1)

In [18]:
ratings_full3 = ratings_full

In [21]:
print(ratings_full.shape)
print(ratings_full3.shape)

(82520, 22)
(82520, 20)


In [22]:
ratings_full3.head(2)

,user id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,63,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [23]:
lst = [] 
for usr in tqdm_notebook(ratings_full3['user id'].unique()):
    tmp = ratings_full3.loc[ratings_full3['user id'] == usr]
    tmplst = []
    tmplst.append(usr)
    sums = tmp.iloc[:,1:20].sum(axis=0)
    for sumamount in sums:
        tmplst.append(sumamount)
    lst.append(tmplst)
cols = [col for col in ratings_full3.columns]
final_FOOKIN_df = pd.DataFrame(lst , columns = cols)

A Jupyter Widget

In [26]:
final_FOOKIN_df.head(3)

,user id,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,196,0,0,1,0,2,27,0,1,11,1,0,0,0,0,9,1,0,1,0
1,63,0,9,6,2,3,26,2,1,36,0,1,1,0,3,18,7,12,4,0
2,226,0,7,4,3,1,11,3,1,25,0,1,0,2,3,4,9,11,5,2


In [28]:
from sklearn import preprocessing

x = final_FOOKIN_df.values #returns a numpy array
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
finaldfnormed3 = pd.DataFrame(x_scaled)

finaldfnormed3 = np.array(finaldfnormed3)

np.save('FOOKIN_df.npy', finaldfnormed3)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [32]:
np.save('FOOKIN_df.npy', finaldfnormed3[:, 1:])